# 1. 데코레이터

# 1.1 데코레이터 함수 정의

### 데코레이터 는 하나의 문법

`new = change_func(my_func)`

In [1]:
def dec(func):
    print("a")
    def new_func():
        print('A')
        return func()
    return new_func

In [2]:
def add() :
    return 10

In [3]:
add = dec(add)

a


In [4]:
add()

A


10

### 데코레이터 단순 표기법 

In [5]:
@dec
def add_() :
    return 100

a


In [6]:
add_()

A


100

## 1.2 데코레이터 실행함수 인자 전달 하기

In [7]:
def dec_1(func):
    print("a")
    def new_func(*args, **kwargs):
        print('A')
        return func(*args, **kwargs)
    return new_func

In [8]:
@dec_1
def add_1(x,y) :
    return x + y

a


In [9]:
add_1(100,200)

A


300

## 1.3  데코레이터 실행순서

In [10]:
def a(func):
    print("a")
    def new_func():
        print('A')
        func()
    return new_func


def b(func):
    print("b")
    def new_func():
        print('B')
        func()
    return new_func


def c(func):
    print("c")
    def new_func():
        print('C')
        func()
    return new_func

### 데코레이터 적용 순서 

In [11]:
@a
@b 
@c # <- 우선 적용
def func():
    pass

c
b
a


### 실행순서 

In [12]:
func()

A
B
C


In [13]:
def func_1():
    pass

func_1 = a(b(c(func_1))) # 데코레이터 문법 적용시키지 않고 동일한 함수를 생성

c
b
a


In [14]:
func_1()

A
B
C


# 2.  정보상실 문제 해결

* 덮어쓰기

`__name__`, `__module__`, `__qualname__`, `__doc__`, `__annotations__`

* 업데이트

`__dict__`

* 원본함수 저장

`__wrapped__`

## 2.1  내부 함수가 전달되어 기존 함수에 대한 정보가 없음

In [15]:
def outer(func):
    def inner(*args, **kwargs):
        print("함수가 실행됩니다.")
        return func(*args, **kwargs)
    return inner

In [16]:
@outer
def func_3(a, b, c):
    """3개의 숫자 또는 Sequence를 받아서 + 연산 후 리턴"""
    return a + b + c

In [17]:
func_3(1, 2, 3)

함수가 실행됩니다.


6

In [18]:
func_3('a', 'b', 'c')

함수가 실행됩니다.


'abc'

In [19]:
func_3.__name__

'inner'

In [20]:
func_3.__doc__

In [21]:
func_3.__module__

'__main__'

In [22]:
func_3.__qualname__

'outer.<locals>.inner'

In [23]:
func_3.__dict__

{}

## 2.2 wraps 구현

In [24]:
from functools import wraps

def dec(func):
    @wraps(func) # 메타 정보 관리
    def wrapper(*args, **kwargs): # *: 가변인자 / **: 키워드인자
        result = func(*args, **kwargs)
        return result
    return wrapper



In [25]:
@dec # @dec 사용하지 않을 경우: add = dec(add) ; def add(x, y): return x + y ;
def add(x, y):
    return x + y

In [26]:
@dec
def multiply(x, y):
    return x * y

In [27]:
print(add(5, 5))

10


In [28]:
add.__name__

'add'

In [29]:
print(multiply(4, 3))

12


In [30]:
multiply.__name__

'multiply'

## 2-3  더 깊은 데코레이터

In [31]:
def make_deco(value):
    def outer(func):
        def inner(*args, **kwargs):
            print(value)
            return func(*args, **kwargs)
        return inner
    return outer

In [32]:
@make_deco('!!!@#!@#!@#!@#$@#$%@!#$%@#$')
def func():
    print("func!!!")

In [33]:
func()

!!!@#!@#!@#!@#$@#$%@!#$%@#$
func!!!


# 3. 데코레이터 활용

### 타입 강제하는 데코레이터

```
@typecheck(int, int, int)
def sum(a, b, c):
    return a + b + c


sum(1, 2, 3) # 6 
sum('a', 'b', 'c') # TypeError
```

### 최상위 테코레이터 함수에 인자를 받아서 처리합니다.

In [34]:
def typecheck(*t_args):# t_args
    def outer(func):
        def inner(*args, **kwargs):
            for i, t_arg in enumerate(t_args):
                if t_arg != type(args[i]):
                    raise TypeError("정확하지 않은 타입입니다.")
            return func(*args, **kwargs)
        return inner
    return outer

In [35]:
@typecheck(int, int, int)
def sum(a, b, c):
    return a + b + c

In [36]:
sum(1, 2, 3)

6

### 최상위 테코레이터 함수에 인자를 받아서 처리합니다.

In [37]:
## 키워드인자들도 반응하도록 

def typecheck_1(*t_args, **t_kwargs):# t_args
    def outer(func):
        def inner(*args, **kwargs):
            # 위치인자 검증
            for i, t_arg in enumerate(t_args):
                if t_arg != type(args[i]):
                    raise TypeError("정확하지 않은 타입입니다.")
                    
            # 키워드인자 검증
            for key in t_kwargs:
                if t_kwargs[key] != kwargs[key].__class__:
                    raise TypeError("정확하지 않은 타입입니다.")
            
            return func(*args, **kwargs)
        return inner
    return outer

In [38]:
@typecheck_1(a=int, b=int, c=int)
def sum(a, b, c):
    return a + b + c

In [39]:
sum(a=3, b=2, c=3)

8

# 4.  클래스 데코레이터

## 4.1 데코레이터로 속성 추가

### 데코레이터 처리 함수 정의 

In [40]:
def class_deco(cls):
    cls.deco = True
    return cls

### 클래스의 속성을 데코레이터에서 처리 

In [41]:
@class_deco
class A:
    pass

### 클래스 속성 확인 

In [42]:
A.deco

True

## 4.2 데코레이터로 메소드 추가 

### `__call__' 속성에 추가될 함수 정의

In [43]:
def sum_(a, b):
    return a + b

### 데코레이터 함수 정의 

In [44]:
def deco_1(cls):
    cls.__call__ = staticmethod(sum_)
    return cls

### 두 개의 클래스를 데코레이터 처리 

In [45]:
@deco_1
class A:
    pass


@deco_1
class B:
    pass

In [46]:
A.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None,
              '__call__': <staticmethod at 0x5070e48>})

In [47]:
B.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'B' objects>,
              '__weakref__': <attribute '__weakref__' of 'B' objects>,
              '__doc__': None,
              '__call__': <staticmethod at 0x5070cc0>})

### 인스턴스를 만들어서 실행

In [48]:
a = A()
b = B()

In [49]:
a(1, 2)

3

In [50]:
b('3', 'a')

'3a'

## 4.3  immutable class만들기


### 데코레이터 함수 정의 

In [51]:
def immutable(cls):
    def new_setattr(self, name, value):
        raise AttributeError("not set attribute ")
    cls.__setattr__ = new_setattr
    return cls

### 클래스 속성만 가진 클래스 정의 

In [52]:
@immutable
class A:
    a = 3
    b = 4

In [53]:
a = A()

In [54]:
a.a, a.b

(3, 4)

In [55]:
try :
    a.a = 10
    a.a
except Exception as e :
    print(e)

not set attribute 


# 5.  인스턴스로 데코레이터 하기

### instance 데코레이터

기능 : 시작전 알림 출력 print

```
deco = InstanceDeco("메세지")

@deco
def func():
    pass
    
func() # "메세지" 출력
```

In [56]:
from functools import wraps

### 인스턴스 데코레이터를 하려면 인스턴스가 실행이 되어야 함 

In [57]:
class InstanceDeco:
    def __init__(self, value):
        self.value = value # __closure__
        
    def __call__(self, func):
        @wraps(func)
        def inner(*args, **kwargs):
            print(self.value)
            return func(*args, **kwargs)
        return inner

In [58]:
deco = InstanceDeco("@@@@")

In [59]:
@deco
def func():
    pass

func()

@@@@


In [60]:
func.__name__

'func'

# 6.  class로  데코레이터 처리하기

기능 : 시작전 알림 출력 print

```

@ClassDeco
def func():
    pass
    
func() # 알림 출력
```

추가 기능 : + wraps 적용

### 클래스 생성할 때 인자로 함수를 받으므로 새로운 객체를  만들고 그 안에 함수를 저장해서 처리

In [61]:
from functools import wraps

In [62]:
class ClassDeco:
    def __new__(cls, *args, **kwargs):
        func = args[0]
        instance = super().__new__(cls)
        return wraps(func)(instance)
    
    def __init__(self, func):
        self.func = func # __closure__ / 저장 in __dict__
        
    
    ## 인스턴스 호출
    def __call__(self, *args, **kwargs):
        print("알림")
#         @wraps(self.func)
#         def inner()
        return self.func(*args, **kwargs)

### `__new__` 내부에 있는 함수를 객체내에 포함하기

In [63]:
def funcX():
    print("wrapped FUNC")

### wraps 함수가 부분 함수이므로 함수를 받아서 처리하고 객체의 인스턴스가 주어지면 이를 받아서 객체로 처리

In [64]:
wraps(funcX)

functools.partial(<function update_wrapper at 0x00000000028B8048>, wrapped=<function funcX at 0x000000000509A510>, assigned=('__module__', '__name__', '__qualname__', '__doc__', '__annotations__'), updated=('__dict__',))

In [65]:
aaa = wraps(funcX)(ClassDeco(funcX))

### 객체 내부의 정보가 변경됨 

In [66]:
aaa.__dict__

{'__module__': '__main__',
 '__name__': 'funcX',
 '__qualname__': 'funcX',
 '__doc__': None,
 '__annotations__': {},
 '__wrapped__': <function __main__.funcX()>,
 'func': <function __main__.funcX()>}

### 인스턴스 내에 저장된 함수가 실행됨

In [67]:
# func = ClassDeco(func)

@ClassDeco
def func():
    print("ORIGIN FUNC")

In [68]:
func()

알림
ORIGIN FUNC


In [69]:
type(func)

__main__.ClassDeco

In [70]:
func.__name__

'func'

# 7. 내장 데코레이터

### lru_cache 를 이용해서 재귀작업에 대한 정보를 보관해서 빨리 처리 

### 재귀호출로만 처리 

In [71]:
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [72]:
%%time
fib(35)

Wall time: 5.62 s


9227465

### 기존 값을 내부 딕셔너리에 저장

In [73]:
def memo(func):
    m = {}
    def inner(n):
        if not m.get(n):
            m[n] = func(n)
        return m[n]
    return inner

In [74]:
@memo
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [75]:
%%time
fib(35)

Wall time: 1 ms


9227465

###  내부의 메모를 처리할 수 있도록 처리

In [76]:
from functools import lru_cache

In [77]:
@lru_cache()
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [78]:
%%time
fib(35)

Wall time: 0 ns


9227465

In [79]:
help(lru_cache)

Help on function lru_cache in module functools:

lru_cache(maxsize=128, typed=False)
    Least-recently-used cache decorator.
    
    If *maxsize* is set to None, the LRU features are disabled and the cache
    can grow without bound.
    
    If *typed* is True, arguments of different types will be cached separately.
    For example, f(3.0) and f(3) will be treated as distinct calls with
    distinct results.
    
    Arguments to the cached function must be hashable.
    
    View the cache statistics named tuple (hits, misses, maxsize, currsize)
    with f.cache_info().  Clear the cache and statistics with f.cache_clear().
    Access the underlying function with f.__wrapped__.
    
    See:  http://en.wikipedia.org/wiki/Cache_algorithms#Least_Recently_Used



# 8. singledispatch

### 하나의 함수에 여러가지 타입을 받아서 다양한 함수를 처리할 수 있음

In [80]:
from functools import singledispatch

In [81]:
help(singledispatch)

Help on function singledispatch in module functools:

singledispatch(func)
    Single-dispatch generic function decorator.
    
    Transforms a function into a generic function, which can have different
    behaviours depending upon the type of its first argument. The decorated
    function acts as the default implementation, and additional
    implementations can be registered using the register() attribute of the
    generic function.



### 함수를 별도로  등록해서 처리 

In [82]:
@singledispatch
def func(arg):
    print("3배 증가")
    print(arg * 3)

In [83]:
func.register

<function functools.singledispatch.<locals>.register(cls, func=None)>

### 타입이 등록된 것 별로 처리 

In [84]:
@func.register(int)
def func_int(arg):
    print("정수가 3배 커집니다.")
    print(arg * 3)

    
@func.register(str)
def func_str(arg):
    print("문자열이 3번 반복됩니다.")
    print(arg * 3)

### 리스트를 넣으면 첫번째 함수를 호출

In [85]:
func([1, 2, 3])

3배 증가
[1, 2, 3, 1, 2, 3, 1, 2, 3]


### 정수를 넣으면 정수로 등록된 함수를 호출 

In [86]:
func(123)

정수가 3배 커집니다.
369


### 문자열을 넣으면 문자열로 등록된 함수를 호출 

In [87]:
func('abcd')

문자열이 3번 반복됩니다.
abcdabcdabcd


### 실제 정수를 호출하지만 타입 체크를 하지 않아 문자도 처리됨 

In [88]:
func.dispatch(int)('asdfasdf')

정수가 3배 커집니다.
asdfasdfasdfasdfasdfasdf
